In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dataset_loader
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from skimage.io import imread_collection

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
batch_size = 100

In [ ]:
def image_plot(image_vector, color):
  size = image_vector.shape[0]
  size = int(size**(1/2))
  f, ax = plt.subplots(size,size)
  for i in range(size):
    for j in range(size):
      if color:
        ax[i,j].imshow(image_vector[i*size+j].astype(np.uint8))
      else:
        ax[i,j].imshow(image_vector[i*size+j,:,:,0].astype(np.uint8), cmap='gray', vmin=0, vmax=255)

In [ ]:
def image_collection_to_numpy( images ):
    images_RGB = np.zeros( (len(images),32,32,3), )

    for i in range( len(images) ):
        images_RGB[i] = images[i] 
    
    return images_RGB.astype(np.float32)

In [ ]:
def transform_to_gray( images_RGB ):

    R = images_RGB[:, :, :,0]
    G = images_RGB[:, :, :,1]
    B = images_RGB[:, :, :,2]

    images_GRAY =  0.299*R  + 0.587*G + 0.114*B 

    return images_GRAY.reshape( images_RGB.shape[0],32,32,1 )

In [ ]:
path_train = './../input/data-pdi/data/train/*.jpg'
path_test = './../input/data-pdi/data/test/*.jpg'

train = imread_collection(path_train)
test = imread_collection(path_test)


In [ ]:
train_RGB = image_collection_to_numpy(train)
test_RGB = image_collection_to_numpy(test)

In [ ]:
def shuffle_images(images_RGB):

    idx = np.arange( train_RGB.shape[0] )
    np.random.shuffle(idx)
    
    aux = np.zeros( train_RGB.shape )
    
    for i in range( train_RGB.shape[0] ):
        aux[ i, :, :, :] = images_RGB[ idx[i], :, :, :]

    return aux

In [ ]:
train_RGB_t = shuffle_images(train_RGB)

In [ ]:
train_GRAY = transform_to_gray(train_RGB)
test_GRAY = transform_to_gray(test_RGB)

In [ ]:
def transform_in_batches(images_GRAY, images_RGB, batch_size):
    #pegando o numero de batches
    img_G_batchs = np.array(  [ images_GRAY[_ix : _ix+batch_size] for _ix in range(0, images_GRAY.shape[0], batch_size)  ]  )
    img_RGB_batchs = np.array(  [ images_RGB[_ix : _ix+batch_size] for _ix in range(0, images_RGB.shape[0], batch_size)  ]  )

    return img_G_batchs, img_RGB_batchs
    
train_GRAY_b, train_RGB_b = transform_in_batches(train_GRAY, train_RGB, batch_size)
test_GRAY_f, test_RGB_f = transform_in_batches( test_GRAY, test_RGB, batch_size )

In [ ]:
def transform_and_create_torch_tensors(data, batch_size):
    
    data = torch.from_numpy(data).float().cuda()
    return data.contiguous().view(data.size(0),batch_size,-1,32,32)

In [ ]:
!nvidia-smi

Mon Jun 21 22:18:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_GRAY_tensor = transform_and_create_torch_tensors(train_GRAY_b, batch_size)
train_RGB_tensor = transform_and_create_torch_tensors(train_RGB_b, batch_size)

test_GRAY_tensor = transform_and_create_torch_tensors(test_GRAY_f, batch_size)
test_RGB_tensor = transform_and_create_torch_tensors(test_RGB_f, batch_size)

In [ ]:
MEAN = torch.mean( train_GRAY_tensor[:,:,0,:,:] )
STD = torch.mean( train_GRAY_tensor[:,:,0,:,:] )

train_GRAY_tensor[:,:,0,:,:] = (train_GRAY_tensor[:,:,0,:,:]-MEAN)/STD
train_RGB_tensor /= 255

test_GRAY_tensor[:,:,0,:,:] = (test_GRAY_tensor[:,:,0,:,:]-MEAN)/STD
test_RGB_tensor /= 255

In [ ]:
def double_conv(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
        nn.ReLU(inplace=True)

    )

    return conv

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = double_conv(1,16)
        self.conv2 = double_conv(16,32)
        self.conv3 = double_conv(32,32)
        self.linear1 = nn.Linear(32*8*8, 512)
        self.linear2 = nn.Linear(512, 3072)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, image):
        #taking batch_size
        batch_size = image.size()[0]
        # Encoder Phase
        x1 = self.conv1(image)
        x2 = self.max_pool(x1)
        
        x3 = self.conv2(x2)
        x4 = self.max_pool(x3)
        
        x5 = self.conv3(x4)
        # Decoder Phase
        x6 = self.linear1(x5.view(batch_size,-1))
        
        f1 = F.relu(x6)
        f2 = torch.sigmoid(self.linear2(f1)).view(-1, 3, 32, 32)
        return f2